In [ ]:
!pip install pyvi

In [ ]:
import torch
from transformers import pipeline
from pyvi import ViTokenizer
import os
from dotenv import load_dotenv
load_dotenv()
MODEL_NAME_SA = os.getenv('MODEL_NAME_SA')
pipe = pipeline("text-classification", model=MODEL_NAME_SA)

In [50]:
def pred_label(text):
    segment_w = ViTokenizer.tokenize(text)
    return pipe(segment_w)[0]['label']
   

In [51]:
import pandas as pd 

cmt_df = pd.read_csv('../../data/results/comments.csv')
cmt_df = cmt_df.drop(columns=['Unnamed: 0', 'Name', 'PName', 'Description'], axis=1)

# rename columns
cmt_df = cmt_df.rename(columns={'Pid': 'pid', 'Time':'time', 'Comment': 'cmt', 'Images': 'img'})

# drop nan
cmt_df = cmt_df.dropna(subset=['cmt'])

# remove duplicates
cmt_df = cmt_df.drop_duplicates()

cmt_df.head()

,pid,time,cmt,img
0,232100001,14: 56 | 14/08/2023,Cây này cây thứ 3 rồi vẫn chưa có dấu hiệu dừn...,[]
1,232100001,16: 51 | 31/10/2023,"Cây son dưỡng mình thích nhất từ trc tới giờ, ...",['https://media.hcdn.vn/rating/169874591132980...
2,232100001,19: 48 | 20/01/2022,Mới từ đầu sẽ thấy mùi son hơi khó ngửi nhưng ...,[]
3,232100001,00: 35 | 18/11/2022,"Giá cả hợp lý, giao hàng nhanh. Sản phẩm không...",[]
4,232100001,17: 42 | 24/08/2024,"Xài mướt, cảm giác nhẹ môi chứ k nặng dàyooi n...",[]


In [52]:
cmt_df.shape

(13454, 4)

In [53]:
cmt_df['cmt_label'] = cmt_df['cmt'].apply(lambda x: pred_label(x))
cmt_df.head()

,pid,time,cmt,img,cmt_label
0,232100001,14: 56 | 14/08/2023,Cây này cây thứ 3 rồi vẫn chưa có dấu hiệu dừn...,[],POS
1,232100001,16: 51 | 31/10/2023,"Cây son dưỡng mình thích nhất từ trc tới giờ, ...",['https://media.hcdn.vn/rating/169874591132980...,POS
2,232100001,19: 48 | 20/01/2022,Mới từ đầu sẽ thấy mùi son hơi khó ngửi nhưng ...,[],POS
3,232100001,00: 35 | 18/11/2022,"Giá cả hợp lý, giao hàng nhanh. Sản phẩm không...",[],POS
4,232100001,17: 42 | 24/08/2024,"Xài mướt, cảm giác nhẹ môi chứ k nặng dàyooi n...",[],POS


In [54]:
cmt_df.to_csv('../../data/results/preprocessed_comments.csv', index=False)

## Concat comment for each (productID, comment label (neg, pos, neu))

In [1]:
import pandas as pd
cmt_df = pd.read_csv('../../data/results/preprocessed_comments.csv')
cmt_df.head()

,pid,time,cmt,img,cmt_label
0,232100001,14: 56 | 14/08/2023,Cây này cây thứ 3 rồi vẫn chưa có dấu hiệu dừn...,[],POS
1,232100001,16: 51 | 31/10/2023,"Cây son dưỡng mình thích nhất từ trc tới giờ, ...",['https://media.hcdn.vn/rating/169874591132980...,POS
2,232100001,19: 48 | 20/01/2022,Mới từ đầu sẽ thấy mùi son hơi khó ngửi nhưng ...,[],POS
3,232100001,00: 35 | 18/11/2022,"Giá cả hợp lý, giao hàng nhanh. Sản phẩm không...",[],POS
4,232100001,17: 42 | 24/08/2024,"Xài mướt, cảm giác nhẹ môi chứ k nặng dàyooi n...",[],POS


In [3]:
# Count occurrences of each cmt_label per pid
label_counts = cmt_df.groupby(['pid', 'cmt_label']).size().unstack(fill_value=0).add_prefix('count_')

# Concatenate comments for each label per Pid
cmt_concat = cmt_df.groupby(['pid', 'cmt_label'])['cmt'].apply(lambda x: ' '.join(x)).unstack(fill_value='').add_prefix('cmt_summary_')

# Merge the count and concatenated columns
cmt_concat = label_counts.join(cmt_concat).reset_index()
cmt_concat


cmt_label,pid,count_NEG,count_NEU,count_POS,cmt_summary_NEG,cmt_summary_NEU,cmt_summary_POS
0,100070011,1,2,7,Dùng thấy da nách đở thâm. Da lưng đở mụn và v...,Mình xem review nên mua về dùng thử xem sao. N...,"xài được 1 thời gian mới lên đánh giá , xài mề..."
1,100070048,1,2,8,khong bị nhờn khi bôi.khi rửa mới thấy nhờn thui,"da sáng hơn, có điều mùi hơi nồng dùng cho mặt...",Xài dòng này sướng hơn veselin Vslin làm mình ...
2,100070053,0,3,10,,hang OK lam moi người nen mua Mình vừa nhận hà...,Sản phẩm tốt đáng sửa dụng Mình quá là ưng cha...
3,100080005,1,0,6,"Dạng gel nhưng không khô nhanh, mùa hè cảm giá...",,"Nhận hàng chuẩn, thơm.Đúng mùi mình đã mua. Lấ..."
4,100080018,1,0,6,"Dạng gel nhưng không khô nhanh, mùa hè cảm giá...",,"Nhận hàng chuẩn, thơm.Đúng mùi mình đã mua. Lấ..."
...,...,...,...,...,...,...,...
1902,422224364,0,0,3,,,Rất tốt gh nhanh chai xanh dương hlc mùi thơm ...
1903,422224365,0,0,3,,,Rất tốt gh nhanh chai xanh dương hlc mùi thơm ...
1904,422226465,0,0,3,,,Rất tốt gh nhanh chai xanh dương hlc mùi thơm ...
1905,422227265,0,0,3,,,Rất tốt gh nhanh chai xanh dương hlc mùi thơm ...


In [4]:
cmt_concat.to_csv('../../data/results/cmt_concat.csv')

In [ ]:
product_df = pd.read_csv('../../data/results/product_data.csv')
product_df = product_df.drop(columns=['Unnamed: 0']).drop_duplicates()
product_df.shape

In [ ]:
# Merge on 'pid' key
result = pd.merge(product_df, cmt_concat, on='pid', how='left')  # how='inner' returns only matching rows
result.to_csv('../../data/results/product_data_v2.csv')


## Comment summary (pending)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="suriya7/bart-finetuned-text-summarization")
max_length = 142
def generate_summary(text):
    if len(text) > max_length:
        return pipe(text)[0]['summary_text']
    else:
        return text
cmt_concat['cmt_summary_NEG'] = cmt_concat['cmt_summary_NEG'].apply(lambda x: generate_summary(x))
cmt_concat['cmt_summary_NEU'] = cmt_concat['cmt_summary_NEU'].apply(lambda x: generate_summary(x))
cmt_concat['cmt_summary_POS'] = cmt_concat['cmt_summary_POS'].apply(lambda x: generate_summary(x))


In [ ]:
cmt_concat.to_csv('../../data/results/summary_cmt.csv')